#Pythonで学ぶ画像認識　第6章 画像キャプショニング
##第6.3節 CNN-LSTMによる手法〜Show and tellを実装してみよう

###モジュールのインポートとGoogleドライブのマウント

In [1]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import datetime
from tqdm import tqdm
import pickle
import random
from torch.utils.data.sampler import SubsetRandomSampler
from PIL import Image
from typing import Sequence, Dict, Tuple, Union
from collections import deque

import torch
from torch import nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence
from torchvision import models
import torchvision.transforms as T
import torchvision.datasets as dataset

from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('drive/MyDrive/python_image_recognition/6_img_captioning/6_3_show_and_tell')

import util

Mounted at /content/drive


### 画像エンコーダの実装

In [2]:
class CNNEncoder(nn.Module):
    '''
    Show and tellのエンコーダ
    dim_embedding: 埋め込み次元
    '''
    def __init__(self, dim_embedding: int):
        super().__init__()

        # ImageNetで事前学習された
        # ResNet152モデルをバックボーンネットワークとする
        resnet = models.resnet152(weights="IMAGENET1K_V2") 

        # 特徴抽出器として使うため全結合層を削除
        modules = list(resnet.children())[:-1]
        self.backbone = nn.Sequential(*modules)

        # デコーダへの出力
        self.linear = nn.Linear(resnet.fc.in_features, dim_embedding)

    '''
    エンコーダの順伝播
    imgs: 入力画像, [バッチサイズ, チャネル数, 高さ, 幅]
    '''
    def forward(self, imgs: torch.Tensor):
        # 特徴抽出 -> [バッチサイズ, 2048]
        # 今回はバックボーンネットワークは学習させない
        with torch.no_grad():
            features = self.backbone(imgs)
            features = features.flatten(1)

        # 全結合
        features = self.linear(features)

        return features

###文生成デコーダの実装

In [3]:
class RNNDecoder(nn.Module):
    '''
    Show and tellのデコーダ
    dim_embedding: 埋め込み次元（単語埋め込み次元）
    dim_hidden   : 隠れ層次元
    vocab_size   : 辞書サイズ
    num_layers   : レイヤー数
    dropout      : ドロップアウト確率
    '''
    def __init__(self, dim_embedding: int, dim_hidden: int, 
                 vocab_size: int, num_layers: int, dropout: int=0.1):
        super().__init__()

        # 単語埋め込み
        self.embed = nn.Embedding(vocab_size, dim_embedding)

        # LSTM
        self.lstm = nn.LSTM(dim_embedding, dim_hidden, 
                            num_layers, batch_first=True)

        # 全結合層
        self.linear = nn.Linear(dim_hidden, vocab_size)

        # ドロップアウト
        self.dropout = nn.Dropout(dropout)

    '''
    デコーダの順伝播
    features: エンコーダ出力特徴, [バッチサイズ, 埋め込み次元]
    captions: 画像キャプション,   [バッチサイズ, 系列長]
    lengths : 系列長のリスト
    '''
    def forward(self, features: torch.Tensor, captions: torch.Tensor,
                lengths: list):

        # 単語埋め込み -> [バッチサイズ, 系列長, 埋め込み次元]
        embeddings = self.embed(captions)

        # 画像埋め込みと単語埋め込みとを連結
        # features.unsqueeze(1) -> [バッチサイズ, 1, 埋め込み次元]
        # 連結後embeddings -> [バッチサイズ, 系列長 + 1, 埋め込み次元]
        embeddings = torch.cat((features.unsqueeze(1), embeddings), 1)
        
        # パディングされたTensorを可変長系列に戻してパック
        # packed.data() -> [実際の系列長, 埋め込み次元]
        packed = pack_padded_sequence(embeddings,
                                      lengths, batch_first=True)

        # LSTM
        hiddens, cell = self.lstm(packed)

        # ドロップアウト
        output = self.dropout(hiddens[0])

        # ロジットを取得
        outputs = self.linear(output)

        return outputs

    '''
    サンプリングによる説明文出力（貪欲法）
    features  : エンコーダ出力特徴, [バッチサイズ, 埋め込み次元]
    states    : LSTM隠れ状態
    max_length: キャプションの最大系列長
    '''
    @torch.no_grad()
    def sample(self, features: torch.Tensor, 
               states=None, max_length=30):

        inputs = features.unsqueeze(1)
        word_idx_list = []

        # 最大系列長まで再帰的に単語をサンプリング予測
        for step_t in range(max_length):
            # LSTM隠れ状態を更新
            hiddens, states = self.lstm(inputs, states)

            # 単語予測
            outputs = self.linear(hiddens.squeeze(1))
            outputs = outputs.softmax(dim=1)
            preds = outputs.argmax(dim=1)
            word_idx_list.append(preds[0].item())
            
            # t+1の入力を作成
            inputs = self.embed(preds)
            inputs = inputs.unsqueeze(1)  

        return word_idx_list

###サンプルからミニバッチを生成するcollate関数

In [4]:
'''
batch     : サンプルした複数の画像とラベルをまとめたもの
word_to_id: 単語->単語ID辞書
'''
def collate_func(batch: Sequence[Tuple[Union[torch.Tensor, str]]],
                 word_to_id: Dict[str, int]):
    imgs, captions = zip(*batch)

    # それぞれのサンプルの5個のキャプションの中から1つを
    # 選択してトークナイズ
    captions = [tokenize_caption(
        random.choice(cap), word_to_id) for cap in captions]

    # キャプションの長さが降順になるように並び替え
    batch = zip(imgs, captions)
    batch = sorted(batch, key=lambda x: len(x[1]), reverse=True)
    imgs, captions = zip(*batch)
    imgs = torch.stack(imgs)

    lengths = [cap.shape[0] for cap in captions]
    targets = torch.full((len(captions), max(lengths)), 
                         word_to_id['<null>'], dtype=torch.int64)
    for i, cap in enumerate(captions):
        end = lengths[i]
        targets[i, :end] = cap[:end]
    
    return imgs, targets, lengths

###トークナイザ

In [5]:
'''
トークナイザ - 文章(caption)を単語IDのリスト(tokens_id)に変換
caption   : 画像キャプション
word_to_id: 単語->単語ID辞書
'''
def tokenize_caption(caption: str, word_to_id: Dict[str, int]):
    tokens = caption.lower().split()
    
    tokens_temp = []    
    # 単語についたピリオド、カンマを削除
    for token in tokens:
        if token == '.' or token == ',':
            continue

        token = token.rstrip('.')
        token = token.rstrip(',')
        
        tokens_temp.append(token)
    
    tokens = tokens_temp        
        
    # 文章(caption)を単語IDのリスト(tokens_id)に変換
    tokens_ext = ['<start>'] + tokens + ['<end>']
    tokens_id = []
    for k in tokens_ext:
        if k in word_to_id:
            tokens_id.append(word_to_id[k])
        else:
            tokens_id.append(word_to_id['<unk>'])
    
    return torch.Tensor(tokens_id)

###学習におけるハイパーパラメータやオプションの設定

In [6]:
class ConfigTrain(object):
    '''
    ハイパーパラメータ、グローバル変数の設定
    '''  
    def __init__(self):

        # ハイパーパラメータ
        self.dim_embedding = 300 # 埋め込み層の次元
        self.dim_hidden = 128     # LSTM隠れ層の次元
        self.num_layers = 2        # LSTM階層の数
        self.lr = 0.001             # 学習率
        self.dropout = 0.3         # dropout確率
        self.batch_size = 30       # ミニバッチ数
        self.num_epochs = 100    # エポック数
        self.lr_drop = [20]         # 学習率を減衰させるエポック
        
        # パスの設定
        self.img_directory = 'val2014'
        self.anno_file = 'drive/MyDrive/python_image_recognition/data/coco2014/captions_val2014.json'
        self.word_to_id_file = 'drive/MyDrive/python_image_recognition/6_img_captioning/vocab/word_to_id.pkl'
        self.save_directory = 'drive/MyDrive/python_image_recognition/6_img_captioning/model'

        # 検証に使う学習セット内のデータの割合
        self.val_ratio = 0.3

        # データローダーに使うCPUプロセスの数
        self.num_workers = 4

        # 学習に使うデバイス
        self.device = 'cuda'

        # 移動平均で計算する損失の値の数
        self.moving_avg = 100

### 学習を行う関数

In [ ]:
def train():
    config = ConfigTrain()

    # 辞書（単語→単語ID）の読み込み
    with open(config.word_to_id_file, 'rb') as f:
        word_to_id = pickle.load(f)

    # 辞書サイズを保存
    vocab_size = len(word_to_id)
        
    # モデル出力用のディレクトリを作成
    os.makedirs(config.save_directory, exist_ok=True)

    # 画像のtransformsを定義
    transforms = T.Compose([
        T.Resize((224, 224)),
        T.RandomHorizontalFlip(),
        T.ToTensor(),
        # ImageNetデータセットの平均と標準偏差
        T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)) 
    ])

    # COCOデータロードの定義
    train_dataset = dataset.CocoCaptions(root=config.img_directory, 
                                         annFile=config.anno_file, 
                                         transform=transforms)
    
    # Subset samplerの生成
    val_set, train_set = util.generate_subset(
        train_dataset, config.val_ratio)

    # 学習時にランダムにサンプルするためのサンプラー
    train_sampler = SubsetRandomSampler(train_set)

    # DataLoaderを生成
    collate_func_lambda = lambda x: collate_func(x, word_to_id)
    train_loader = torch.utils.data.DataLoader(
                        train_dataset, 
                        batch_size=config.batch_size, 
                        num_workers=config.num_workers, 
                        sampler=train_sampler,
                        collate_fn=collate_func_lambda)
    val_loader = torch.utils.data.DataLoader(
                        train_dataset, 
                        batch_size=config.batch_size, 
                        num_workers=config.num_workers, 
                        sampler=val_set,
                        collate_fn=collate_func_lambda)

    # モデルの定義
    encoder = CNNEncoder(config.dim_embedding)
    decoder = RNNDecoder(
        config.dim_embedding, config.dim_hidden, vocab_size, 
        config.num_layers, config.dropout)
    encoder.to(config.device)
    decoder.to(config.device)
    
    # 損失関数の定義
    loss_func = lambda x, y: F.cross_entropy(
        x, y, ignore_index=word_to_id.get('<null>', None))
    
    # 最適化手法の定義
    params = list(decoder.parameters()) \
             + list(encoder.linear.parameters())
    optimizer = torch.optim.AdamW(params, lr=config.lr)

    # 学習率スケジューラの定義
    scheduler = torch.optim.lr_scheduler.MultiStepLR(
                    optimizer, milestones=config.lr_drop, gamma=0.1)
    
    # 学習経過の書き込み
    now = datetime.datetime.now()
    train_loss_file = '{}/6-3_train_loss_{}.csv'\
        .format(config.save_directory, now.strftime('%Y%m%d_%H%M%S'))
    val_loss_file = '{}/6-3_val_loss_{}.csv'\
        .format(config.save_directory, now.strftime('%Y%m%d_%H%M%S'))

    # 学習
    val_loss_best = float('inf')
    for epoch in range(config.num_epochs):
        with tqdm(train_loader) as pbar:
            pbar.set_description(f'[エポック {epoch + 1}]')

            # 学習モードに設定
            encoder.train()
            decoder.train()

            train_losses = deque()
            for imgs, captions, lengths in pbar:
                # ミニバッチを設定
                imgs = imgs.to(config.device)
                captions = captions.to(config.device)

                optimizer.zero_grad()

                # エンコーダ-デコーダモデル
                features = encoder(imgs)
                outputs = decoder(features, captions, lengths)

                # ロスの計算
                targets = pack_padded_sequence(captions, 
                                               lengths, 
                                               batch_first=True)[0]
                loss = loss_func(outputs, targets)

                # 誤差逆伝播
                loss.backward()
                
                optimizer.step()

                # 学習時の損失をログに書き込み
                train_losses.append(loss.item())
                if len(train_losses) > config.moving_avg:
                    train_losses.popleft()
                pbar.set_postfix({
                    'loss': torch.Tensor(train_losses).mean().item()})
                with open(train_loss_file, 'a') as f:
                    print(f'{epoch}, {loss.item()}', file=f)

        # 検証
        with tqdm(val_loader) as pbar:
            pbar.set_description(f'[検証]')

            # 評価モード
            encoder.eval()
            decoder.eval()

            val_losses = []
            for imgs, captions, lengths in pbar:

                # ミニバッチを設定
                imgs = imgs.to(config.device)
                captions = captions.to(config.device)

                # エンコーダ-デコーダモデル
                features = encoder(imgs)
                outputs = decoder(features, captions, lengths)

                # ロスの計算
                targets = pack_padded_sequence(captions, 
                                               lengths, 
                                               batch_first=True)[0]
                loss = loss_func(outputs, targets)
                val_losses.append(loss.item())

                # Validation Lossをログに書き込み
                with open(val_loss_file, 'a') as f:
                    print(f'{epoch}, {loss.item()}', file=f)

        # Loss 表示
        val_loss = np.mean(val_losses)
        print(f'Validation loss: {val_loss}')

        # より良い検証結果が得られた場合、モデルを保存
        if val_loss < val_loss_best:
            val_loss_best = val_loss

            # エンコーダモデルを保存
            torch.save(
                encoder.state_dict(),
                f'{config.save_directory}/6-3_encoder_best.pth')

            # デコーダモデルを保存
            torch.save(
                decoder.state_dict(),
                f'{config.save_directory}/6-3_decoder_best.pth')

###学習データの解凍

In [ ]:
!unzip drive/MyDrive/python_image_recognition/data/coco2014/val2014.zip

ストリーミング出力は最後の 5000 行に切り捨てられました。
 extracting: val2014/COCO_val2014_000000535608.jpg  
 extracting: val2014/COCO_val2014_000000279145.jpg  
 extracting: val2014/COCO_val2014_000000243495.jpg  
 extracting: val2014/COCO_val2014_000000225791.jpg  
 extracting: val2014/COCO_val2014_000000428067.jpg  
 extracting: val2014/COCO_val2014_000000059383.jpg  
 extracting: val2014/COCO_val2014_000000558498.jpg  
 extracting: val2014/COCO_val2014_000000376123.jpg  
 extracting: val2014/COCO_val2014_000000191304.jpg  
 extracting: val2014/COCO_val2014_000000171062.jpg  
 extracting: val2014/COCO_val2014_000000014549.jpg  
 extracting: val2014/COCO_val2014_000000121152.jpg  
 extracting: val2014/COCO_val2014_000000276596.jpg  
 extracting: val2014/COCO_val2014_000000029431.jpg  
 extracting: val2014/COCO_val2014_000000036349.jpg  
 extracting: val2014/COCO_val2014_000000502055.jpg  
 extracting: val2014/COCO_val2014_000000438848.jpg  
 extracting: val2014/COCO_val2014_000000364973.jpg  
 extracting: v

###学習の実行

In [ ]:
train()

loading annotations into memory...
Done (t=1.67s)
creating index...
index created!


Downloading: "https://download.pytorch.org/models/resnet152-f82ba261.pth" to /root/.cache/torch/hub/checkpoints/resnet152-f82ba261.pth


  0%|          | 0.00/230M [00:00<?, ?B/s]

[検証]: 100%|██████████| 406/406 [00:31<00:00, 13.01it/s]


Validation loss: 3.8560754242788984


...

Validation loss: 2.471231885437895


[検証]: 100%|██████████| 406/406 [00:31<00:00, 12.82it/s]


Validation loss: 2.4756299638983066


[検証]: 100%|██████████| 406/406 [00:31<00:00, 12.77it/s]


Validation loss: 2.4759621620178223


###デモにおけるハイパーパラメータやオプションの設定

In [7]:
class ConfigDemo(object):
    '''
    ハイパーパラメータ、グローバル変数の設定
    '''  
    def __init__(self):

        # ハイパーパラメータ
        self.dim_embedding = 300   # 埋め込み層の次元
        self.dim_hidden = 128      # LSTM隠れ層の次元
        self.num_layers = 2        # LSTM階層の数
        
        # パスの設定
        # 画像キャプショニング推論
        self.id_to_word_file = 'drive/MyDrive/python_image_recognition/6_img_captioning/vocab/id_to_word.pkl'
        self.img_dirirectory = 'drive/MyDrive/python_image_recognition/data/image_captioning/'    
        self.save_directory = 'drive/MyDrive/python_image_recognition/6_img_captioning/model'

        # 学習に使うデバイス
        self.device = 'cuda'

###デモを行う関数

In [8]:
def demo():
    config = ConfigDemo()

    # 辞書（単語ID→単語）の読み込み
    with open(config.id_to_word_file, 'rb') as f:
        id_to_word = pickle.load(f)

    # 辞書サイズを保存
    vocab_size = len(id_to_word)
    
    # 画像のtransformsを定義
    transforms = T.Compose([
        T.Resize((224, 224)),
        T.ToTensor(),
        # ImageNetデータセットの平均と標準偏差
        T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)) 
    ])
    
    # エンコーダモデルの定義
    encoder = CNNEncoder(config.dim_embedding)
    encoder.to(config.device)
    encoder.eval()

    # デコーダモデルの定義
    decoder = RNNDecoder(config.dim_embedding, config.dim_hidden, 
                         vocab_size, config.num_layers)
    decoder.to(config.device)
    decoder.eval()

    # モデルの学習済み重みパラメータをロード
    encoder.load_state_dict(
        torch.load(f'{config.save_directory}/6-3_encoder_best.pth'))
    decoder.load_state_dict(
        torch.load(f'{config.save_directory}/6-3_decoder_best.pth'))

    # ディレクトリ内の画像を対象としてキャプショニング実行
    for img_file in sorted(
        glob.glob(os.path.join(config.img_dirirectory, '*.jpg'))):

        # 画像読み込み
        img = Image.open(img_file)
        img = transforms(img)
        img = img.unsqueeze(0)
        img = img.to(config.device)

        # エンコーダ・デコーダモデルによる予測
        feature = encoder(img)
        sampled_ids = decoder.sample(feature)

        # 入力画像を表示
        img_plt = Image.open(img_file)
        img_plt = img_plt.resize([224, 224], Image.LANCZOS)
        plt.imshow(img_plt)
        plt.axis('off')
        plt.show()
        print(f'入力画像: {os.path.basename(img_file)}')

        # 画像キャプショニングの実行
        sampled_caption = []
        for word_id in sampled_ids:
            word = id_to_word[word_id]
            sampled_caption.append(word)
            if word == '<end>':
                break
        
        sentence = ' '.join(sampled_caption)
        print(f'出力キャプション: {sentence}')

        # 推定結果を書き込み
        gen_sentence_out = img_file[:-4] + "_show_and_tell.txt"
        with open(gen_sentence_out, 'w') as f:
            print(sentence, file=f)

###デモの実行

In [9]:
demo()

Output hidden; open in https://colab.research.google.com to view.